In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install heartpy
!pip install pyhrv
!pip install neurokit2
!pip install statsmodels
!pip install biosppy
!pip install tqdm
!pip install phate
!pip install pyod
!pip install dataheroes

In [ ]:
import pandas as pd
import heartpy as hp
from heartpy.analysis import calc_fd_measures
import scipy
from scipy.stats import kurtosis
from scipy.stats import skew
from heartpy.analysis import clean_rr_intervals
from heartpy.analysis import calc_ts_measures
import pyhrv.tools as tools
from pyhrv.hrv import hrv
import neurokit2 as nk
import numpy as np
import statsmodels.api as sm
from scipy.signal.signaltools import wiener
import biosppy
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.utils import pad_sequences
import pywt


class Extractor:
    def __init__(self, x):
        self.X = x
        
    def extract(self):
        vals = []
        for index in tqdm(range(0, self.X.shape[0])):
            maxlen = self.X.shape[1] - 9669
            #padded_row = pad_sequences(self.X.loc[index, :], maxlen=maxlen, dtype=float, padding='post', truncating = 'post', value=self.X.loc[index, maxlen - 1])
            row = self.X.loc[index, :]
            res = self._extract_one(row)
            vals.append(res)

        res = pd.DataFrame.from_records(vals)

        return res
        
    def _extract_one(self, signal):
        cur_row = {}

        # Automatically process the (raw) ECG signal
        cleaned_ecg = nk.ecg_clean(signal, sampling_rate=300, method='biosppy')
        cleaned_ecg = hp.remove_baseline_wander(cleaned_ecg, 300)
        cleaned_ecg = nk.signal_detrend(cleaned_ecg)

        rpeaks = biosppy.signals.ecg.hamilton_segmenter(cleaned_ecg, 300)
        rpeaks = biosppy.signals.ecg.correct_rpeaks(cleaned_ecg, rpeaks[0], 300.0)[0]

        r_vals = [cleaned_ecg[r] for r in rpeaks if not np.isnan(r)]
        for i, val in enumerate(r_vals):
            cur_row[f"r{i}"] = val
        
        # Delineate the ECG signal
        _, waves_dwt = nk.ecg_delineate(cleaned_ecg, sampling_rate=300, method="dwt")
        p_onsets = waves_dwt["ECG_P_Onsets"]
        p_offsets = waves_dwt["ECG_P_Offsets"]
        p_peaks = waves_dwt["ECG_P_Peaks"]

        s_peaks = waves_dwt["ECG_S_Peaks"]

        q_peaks = waves_dwt["ECG_Q_Peaks"]

        r_onsets = waves_dwt["ECG_R_Onsets"]
        r_offsets = waves_dwt["ECG_R_Offsets"]

        t_onsets = waves_dwt["ECG_T_Onsets"]
        t_offsets = waves_dwt["ECG_T_Offsets"]
        t_peaks = waves_dwt["ECG_T_Peaks"]

        n_heartbeats = np.size(rpeaks)

        #rr_distance
        rr = (rpeaks[1:] - rpeaks[:n_heartbeats - 1])  # rr-rate in seconds
        cur_row["mean_rr"] = np.nanmean(rr)
        # cur_row["median_rr"] = np.nanmedian(rr)
        cur_row["var_rr"] = np.nanvar(rr)
        cur_row["max_rr"] = np.nanmax(rr)
        cur_row["min_rr"] = np.nanmin(rr)
        cur_row["skew_rr"] = skew(rr)
        cur_row["kurtosis_rr"] = kurtosis(rr)

        #r_amplitude
        r_amplitude = cleaned_ecg[rpeaks]

        cur_row["mean_r"] = np.nanmean(r_amplitude)
        cur_row["median_r"] = np.nanmedian(r_amplitude)
        cur_row["var_r"] = np.nanvar(r_amplitude)
        cur_row["max_r"] = np.nanmax(r_amplitude)
        cur_row["min_r"] = np.nanmin(r_amplitude)

        cur_row["skew_r"] = skew(r_amplitude)
        cur_row["kurtosis_r"] = kurtosis(r_amplitude)

        # q_amplitude
        q_amplitude = q_peaks
        cur_row["mean_q"] = np.nanmean(q_amplitude)
        cur_row["median_q"] = np.nanmedian(q_amplitude)
        cur_row["var_q"] = np.nanvar(q_amplitude)
        cur_row["max_q"] = np.nanmax(q_amplitude)
        cur_row["min_q"] = np.nanmin(q_amplitude)
        cur_row["skew_q"] = skew(q_amplitude)
        cur_row["kurtosis_q"] = kurtosis(q_amplitude)

        # t_amplitude
        t_amplitude = t_peaks
        cur_row["mean_q"] = np.nanmean(t_amplitude)
        cur_row["median_q"] = np.nanmedian(t_amplitude)
        cur_row["var_t"] = np.nanvar(t_amplitude)
        cur_row["max_t"] = np.nanmax(t_amplitude)
        cur_row["min_t"] = np.nanmin(t_amplitude)
        cur_row["skew_t"] = skew(t_amplitude)
        cur_row["kurtosis_t"] = kurtosis(t_amplitude)

        # s_amplitude
        s_amplitude = s_peaks
        cur_row["mean_q"] = np.nanmean(s_amplitude)
        cur_row["median_q"] = np.nanmedian(s_amplitude)
        cur_row["var_s"] = np.nanvar(s_amplitude)
        cur_row["max_s"] = np.nanmax(s_amplitude)
        cur_row["min_s"] = np.nanmin(s_amplitude)
        cur_row["skew_s"] = skew(s_amplitude)
        cur_row["kurtosis_s"] = kurtosis(s_amplitude)

        #qrs_duration
        qrs_duration = [(b - a) for a, b in zip(r_onsets, r_offsets)]
        cur_row["mean_qrs"] = np.nanmean(qrs_duration)
        cur_row["median_qrs"] = np.nanmedian(qrs_duration)
        cur_row["var_qrs"] = np.nanvar(qrs_duration)
        cur_row["max_qrs"] = np.nanmax(qrs_duration)
        cur_row["min_qrs"] = np.nanmin(qrs_duration)
        cur_row["skew_qrs"] = skew(qrs_duration)
        cur_row["kurtosis_qrs"] = kurtosis(qrs_duration)

        #hrv metrics
        hrv_time = nk.hrv_time(rpeaks, sampling_rate=300, show=False)
        cur_row["HRV_IQRNN"] = hrv_time["HRV_IQRNN"].iloc[0]
        cur_row["HRV_HTI"] = hrv_time["HRV_HTI"].iloc[0]
        cur_row["HRV_pNN50"] = hrv_time["HRV_pNN50"].iloc[0]

        # cur_row["HRV_SDNN"] = hrv_time["HRV_SDNN"].iloc[0]
        # cur_row["HRV_RMSSD"] = hrv_time["HRV_RMSSD"].iloc[0]
        # cur_row["HRV_SDSD"] = hrv_time["HRV_SDSD"].iloc[0]
        # cur_row["HRV_CVNN"] = hrv_time["HRV_CVNN"].iloc[0]
        # cur_row["HRV_MedianNN"] = hrv_time["HRV_MedianNN"].iloc[0]
        # cur_row["HRV_pNN50"] = hrv_time["HRV_pNN50"].iloc[0]
        # cur_row["HRV_pNN20"] = hrv_time["HRV_pNN20"].iloc[0]
        # cur_row["HRV_TINN"] = hrv_time["HRV_TINN"].iloc[0]
        
        # FFT
        fft = scipy.fft.fft(cleaned_ecg)
        S = np.abs(fft)/len(cleaned_ecg) #careful, gives large numbers If small numbers are desired then /len(cleaned_ecg)
        S = S[:int(len(S)/2)]
        cur_row["fft_max"] = np.max(S)
        cur_row["fft_sum"] = np.sum(S)
        cur_row["fft_mean"] = np.mean(S)
        cur_row["fft_var"] = np.var(S)
        cur_row["fft_peak"] = np.max(S)
        cur_row["fft_skew"] = skew(S)
        cur_row["fft_kurtosis"] = kurtosis(S)
        
        # Wavelet
        fs = len(cleaned_ecg)
        dist = np.nanmax(rr)
        scales = range(1, 10*dist) #change dist coeff to change number of wavelets, 10 runs in 6 min
        waveletname = 'morl'
        coeff, freq = pywt.cwt(cleaned_ecg, scales, waveletname, 1)
        energies = np.sum(np.square(np.abs(coeff)), axis=-1)
        
        for i, val in enumerate(energies):
            cur_row[f"wvt_energy_{i}"] = val
    
        return cur_row


In [ ]:
import numpy as np
import phate
import umap
from matplotlib import pyplot as plt
from mlxtend.plotting.pca_correlation_graph import corr2_coeff
from pyod.models.ecod import ECOD
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest
from sklearn.impute import SimpleImputer
from sklearn.linear_model import BayesianRidge, LassoCV, Lasso, LogisticRegression
from sklearn.preprocessing import RobustScaler, MinMaxScaler, PolynomialFeatures, StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.feature_selection import f_regression, SelectKBest, chi2, VarianceThreshold, RFE
from dataheroes import CoresetTreeServiceDTC
from sklearn.linear_model import LinearRegression
from scipy.stats import f_oneway
from sklearn.neighbors import LocalOutlierFactor
from sklearn.covariance import EllipticEnvelope

import pandas as pd
pd.DataFrame.iteritems = pd.DataFrame.items


def preprocess(X_train: np.array, y_train: np.array, X_test: np.array, drop_r: bool):
    if drop_r:
        X_train.drop([f"r{r}" for r in range(0, 160)], axis=1, inplace=True)

    X_train.dropna(axis=1, how='all', inplace=True)
    X_test = X_test[X_train.columns]

    X_train.replace(np.inf,np.nan,inplace=True)
    X_test.replace(np.inf,np.nan,inplace=True)

    X_train, X_test = impute_mv(X_train, X_test, 'median')
    X_train, X_test = select_features(X_train, y_train, X_test)
    X_train, X_test = scale_data(X_train, X_test, 'standard')

    return X_train, y_train, X_test


def make_polynomial(X_train: np.array, y_train: np.array, X_test: np.array, degree: int = 2):
    if degree > 2:
        raise Exception("make_polynomial: Insane degree.")

    poly = PolynomialFeatures(2)
    X_train = poly.fit_transform(X_train, y_train)
    X_test = poly.transform(X_test)

    print(X_train.shape)
    return X_train, X_test


def select_features(X_train: np.array, y_train: np.array, X_test: np.array):
    print(X_train.shape)
    X_train, X_test = remove_correlated(X_train, X_test)

    # fs = SelectKBest(score_func=f_regression, k=200)
    # X_train = fs.fit_transform(X_train, y_train.ravel())
    # X_test = fs.transform(X_test)
    # X_train, X_test = recursive_elemination(X_train, y_train, X_test)

    print(X_train.shape)
    return X_train, X_test


def recursive_elemination(X_train: np.array, y_train: np.array, X_test: np.array):
    model = LogisticRegression()
    rfe = RFE(model)

    X_train = rfe.fit_transform(X_train, y_train)
    X_test = rfe.transform(X_test)

    return X_train, X_test

def remove_correlated(X_train: np.array, X_test: np.array):
    # Constant features
    var_threshold = VarianceThreshold(threshold=0.0)  # TODO threshold = 0 for constant
    var_threshold.fit_transform(X_train)
    var_threshold.transform(X_test)

    # Correlated
    cor = corr2_coeff(X_train.T, X_train.T)
    p = np.argwhere(np.triu(np.isclose(cor, 1), 1))
    X_train = np.delete(X_train, p[:, 1], axis=1)
    X_test = np.delete(X_test, p[:, 1], axis=1)
    return X_train, X_test


def scale_data(X_train: np.array, X_test: np.array, method: str = 'min_max'):
    if method == 'robust':
        transformer = RobustScaler()
    elif method == 'min_max':
        transformer = MinMaxScaler()
    elif method == 'standard':
        transformer = StandardScaler()
    else:
        raise Exception(f"Scale: {method} is not implemented")

    X_train = transformer.fit_transform(X_train)
    X_test = transformer.transform(X_test)
    return X_train, X_test


def impute_mv(X_train: np.array, X_test: np.array, method: str = 'iterative'):
    if method == 'median':
        imp = SimpleImputer(missing_values=np.nan, strategy='median')

    elif method == 'mean':
        imp = SimpleImputer(missing_values=np.nan, strategy='mean')

    elif method == 'iterative':
        imp = IterativeImputer(estimator=BayesianRidge(), n_nearest_features=None, imputation_order='ascending')

    else:
        raise Exception(f"Impute: {method} is not implemented")

    X_train = imp.fit_transform(X_train)
    X_test = imp.transform(X_test)

    return X_train, X_test


def detect_remove_outliers(X_train: np.array, y_train: np.array, X_test: np.array):
    train_pred1 = detect_outlier_obs(X_train, y_train, 'coresets')
    train_pred2 = detect_outlier_obs(X_train, X_test, 'ECOD')
    train_pred3 = detect_outlier_obs(X_train, X_test, 'elliptic')
    train_pred4 = detect_outlier_obs(X_train, X_test, 'local_factor')

    train_pred = train_pred1 + train_pred2 + train_pred3 + train_pred4
    train_pred = train_pred > 1

    print(X_train.shape)
    X_train = X_train[train_pred]
    print(X_train.shape)
    y_train = y_train[train_pred]

    return X_train, y_train, X_test


def detect_outlier_obs(X_train: np.array, y_train: np.array, method: str = 'elliptic'):
    train_pred, test_pred = [], []
    if method == 'ECOD':
        ecod = ECOD(contamination=0.03)
        ecod.fit(X_train)
        train_pred = np.array(ecod.labels_) == 0

    elif method == 'isolation_forest':
        for i in range(X_train.shape[1]):
            clf = IsolationForest(n_estimators=150, max_samples='auto', contamination=float(0.03))
            y_train_pred = np.array(clf.fit_predict(X_train[:, i].reshape(-1, 1))) == 1
            train_pred.append(y_train_pred)
            # y_test_pred = np.array(clf.predict(X_test[:, i].reshape(-1, 1))) == 1
            # test_pred.append(y_test_pred)

        train_pred = np.array(train_pred).sum(axis=1)
        # test_pred = np.array(test_pred).sum(axis=1)

    elif method == "coresets":
        tree = CoresetTreeServiceDTC(optimized_for='cleaning')
        tree = tree.build(X=X_train, y=y_train, chunk_size=-1)
        result = tree.get_cleaning_samples(20)
        tree.remove_samples(result['idx'])
        res = tree.get_cleaning_samples(1212)
        # print(res['idx'].shape)
        # print(res["idx"])
        train_pred = np.full((X_train.shape[0],), False)
        train_pred[res["idx"]] = True

    elif method == "local_factor":
        lo = LocalOutlierFactor(n_neighbors=2)
        res = lo.fit_predict(X_train)
        train_pred = np.array(res) == 0

    elif method == "elliptic":
        ee = EllipticEnvelope(random_state=42)
        res = ee.fit_predict(X_train, y_train)
        train_pred = np.array(res) == 0

    else:
        raise Exception(f"Detect: {method} is not implemented")

    return train_pred.astype(int)

In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from tqdm import tqdm
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
import lightgbm as lgb
import tensorflow.keras.layers as layers


def read_data(X_train_path, y_train_path, X_test_path, extract_data):  
    X_train = pd.read_csv(X_train_path)
    X_test = pd.read_csv(X_test_path)
    y_train = pd.read_csv(y_train_path).iloc[:,1]

    if extract_data:
        train_ids, test_ids = X_train.iloc[:, 0], X_test.iloc[:, 0]
        X_train, X_test = X_train.iloc[:,1:], X_test.iloc[:,1:]
    else:
        train_ids, test_ids = pd.DataFrame(list(range(0, X_train.shape[0]))), pd.DataFrame(list(range(0, X_test.shape[0])))

    return X_train, y_train, train_ids, X_test, test_ids


def get_splits(X_train: np.array, y_train: np.array, nfolds: int = 10):
    kf = StratifiedKFold(n_splits=nfolds, random_state=42, shuffle=True)
    return kf.split(X_train, y_train)


def get_model(method: int = 3):
    if method == 1:
        model = XGBClassifier()

    elif method == 2:
        model = CatBoostClassifier(iterations=1000, learning_rate=0.01, logging_level='Silent')

    elif method == 3:
        estimators = [ 
            ('cb', CatBoostClassifier(iterations=1000, learning_rate=0.01, logging_level='Silent')),
            ('xgb', XGBClassifier(random_state=42)),
            # ('lgbm', lgb.LGBMClassifier(random_state=42))
        ]
    
        model = StackingClassifier(estimators=estimators, final_estimator=CatBoostClassifier(iterations=1000, learning_rate=0.01, logging_level='Silent'))
    elif method == 4:
        model = tf.keras.Sequential([keras.Input(shape=(X.shape[1],)),
                                    layers.Dense(2000, activation='relu'),
                                    layers.Dense(1250, activation='relu'),
                                    layers.Dense(500, activation='relu'),
                                    layers.Dense(250, activation='relu'),
                                    layers.Dense(3, activation='relu')])
        model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True), 
                      optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)) #Adam, SGD or adamax

    return model


def main():
    extract_data = True
    nn = False
    method = 3
    if nn:
        method = 4
        
    # read data
    if extract_data:
        X_train_path, y_train_path, X_test_path = "/kaggle/input/aml-ecg/X_train.csv", "/kaggle/input/aml-ecg/y_train.csv", "/kaggle/input/aml-ecg/X_test.csv"

    else:
        X_train_path, y_train_path, X_test_path = "data/train_feat.csv", "data/y_train.csv", "data/test_feat.csv"

    X_train, y_train, train_ids, X_test, test_ids = read_data(X_train_path, y_train_path, X_test_path, extract_data)

    # extract
    if extract_data:
        extr = Extractor(X_train)
        train_feat = extr.extract()
        X_train = train_feat
        train_feat.to_csv("train_feat.csv",index=False)

        extr = Extractor(X_test)
        test_feat = extr.extract()
        X_test = test_feat
        test_feat.to_csv("test_feat.csv",index=False)
    
    print("Extracted / read data.")

    X_train, y_train, X_test = preprocess(X_train, y_train, X_test, drop_r=False)

    print("Preprocessed.")
    
    nfolds = 5
    splits = get_splits(X_train, y_train, nfolds)
    
    model = get_model(method)
    f1_scores = 0
    for i, (train_index, test_index) in enumerate(splits):
        model = get_model(method)
        if nn:
            model.fit(X_train[train_index], y_train[train_index], epochs=75)
        else:
            model.fit(X_train[train_index], y_train[train_index])

        pred = model.predict(X_train[test_index])

        score = f1_score(y_train[test_index], pred, average="micro")

        print(f"Fold {i}: score {score}")
        f1_scores += score

    print(f"Avg F1: {f1_scores / nfolds}")

    model_full = get_model(method)
    if nn:
        model_full.fit(X_train,y_train, epochs=75)
    else:
        model_full.fit(X_train,y_train)
        
    
    res = model_full.predict(X_test)

    out = pd.DataFrame()
    out["id"] = test_ids.iloc[:, 0]
    out["y"] = res

    out.to_csv("data/out.csv", index=False)


main()